<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Получение-и-обзор-данных" data-toc-modified-id="Получение-и-обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Получение и обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li></ul></li><li><span><a href="#Формирование-новых-признаков" data-toc-modified-id="Формирование-новых-признаков-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Формирование новых признаков</a></span><ul class="toc-item"><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Установление-зависимостей-целевого-признака-от-признаков-данных" data-toc-modified-id="Установление-зависимостей-целевого-признака-от-признаков-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Установление зависимостей целевого признака от признаков данных</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков


Входные данные от банка — статистика о платёжеспособности клиентов.

Гипотеза: семейное положение и количество детей клиента влияет на факт погашения кредита в срок.

Цель: проверить выдвинутую гипотезу. Если гипотеза подтвердится: установить, каким образом семейное положение и количество детей клиента влияет на факт погашения кредита в срок.

Также, необходимо проверить влияние иных факторов на факт погашения кредита в срок.

## Получение и обзор данных

In [1]:
import pandas as pd
from nltk.stem import SnowballStemmer 
from pymystem3 import Mystem

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
print('Информация о количестве и типе данных в таблице:')
data.info()

Информация о количестве и типе данных в таблице:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
print('Первые пять строк таблицы:')
display(data.head())

Первые пять строк таблицы:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
print('Последние пять строк таблицы:')
display(data.tail())

Последние пять строк таблицы:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Описание столбцов:

`children` — количество детей в семье

`days_employed` — общий трудовой стаж в днях

`dob_years` — возраст клиента в годах

`education` — уровень образования клиента

`education_id` — идентификатор уровня образования

`family_status` — семейное положение

`family_status_id` — идентификатор семейного положения

`gender` — пол клиента

`income_type` — тип занятости

`debt` — имел ли задолженность по возврату кредитов

`total_income` — ежемесячный доход

`purpose` — цель получения кредита

**Вывод** 

Всего в таблице 12 стобцов и 3 различных типа данных.
Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.

Таблица несет в себе информацию о заемщиках банка, определяющей для нас является информация о задолжности перед банком.

Обработаем пропущенные значения, дубликаты и некорректные значения в таблице. Заменим тип данных в столбцах "общий трудовой стаж" и "ежемесячный доход". Сгруппируем данные, исходя из поставленной задачи. После этого установим зависимость между количеством детей и возвратом кредита в срок, уровнем дохода и возвратом кредита в срок, целью получения кредита и возвратом кредита в срок, семейным положением и возвратом кредита в срок.


## Предобработка данных

### Обработка пропусков

In [6]:
print('Количество пропусков:')
display(data.isna().sum())

Количество пропусков:


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски в столбце `days_employed` и `total_income`.



In [7]:
display(data[data['days_employed'].isna()].head())
print()
display(data[data['total_income'].isna()].head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [8]:
display(data['days_employed'].value_counts())

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

Так же обнаружен отрицательный трудовой стаж, чего быть не может. Уберем отрицательный знак перед значениями.

In [9]:
data['days_employed'] = abs(data['days_employed'])

In [10]:
display(data['total_income'].value_counts())

169846.427535    1
257737.077768    1
200508.675866    1
106196.235958    1
248730.171354    1
                ..
175057.266090    1
101516.604975    1
239154.168013    1
165009.733021    1
189255.286637    1
Name: total_income, Length: 19351, dtype: int64

 Заменим пропущенные значения столбца `days_employed` на ноль. 

In [11]:
data['days_employed'] = data['days_employed'].fillna(0)

In [12]:
display(data.isna().sum())

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Для замены пропусков в столбце `total_income`, найдем характерные значения других столбцов в тех строках, где присутствуют пропуски. На основании этих характерных значений произведем замену. Вычислять будем медианные значения, для меньшего воздействия выбросов.

In [13]:
print(data[data['total_income'].isna() == True]['education_id'].median())

1.0


In [14]:
print(data[data['total_income'].isna() == True]['children'].median())

0.0


In [15]:
print(data[data['total_income'].isna() == True]['family_status_id'].median())

0.0


In [16]:
print(data['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


В столбце `dob_years` обнаружены значения ноль, удалим строки, с таким значением.

In [17]:
data = data[data['dob_years'] != 0]

In [18]:
print(data[data['total_income'].isna() == True]['dob_years'].median())


43.0


In [19]:
total_income_isna_median = data[
    (data['education_id'] == 1) & 
    (data['family_status_id'] == 1) & 
    (data['children'] <= 1) & 
    (data['dob_years'] > 33) &
    (data['dob_years'] < 53)]['total_income'].median().round()



In [20]:
data['total_income'] = data['total_income'].fillna(total_income_isna_median)


**Вывод**

Пропущенные значения обнаружены в столбцах "общий трудовой стаж" и "ежемесячный доход". Тип пропущенных значений в обоих столбцах - NaN, это говорит о том, что в ячейке находится «не число». Номера строк с пропущенными значениями в обоих столбцах совпадают. Это говорит о том, что расчет данных этих столбцов связан друг с другом. Скорее всего при формировании таблицы в некоторых случаях в столбец total_income пытались внести строковые значения.
Чтобы не потерять данные, в первом случае, пропущенные значения заменены нулями, во втором случае, медианным значением, вычисленным по сгруппрованным данным. 

### Замена типа данных

In [21]:
print('Типы данных до изменения:')
data.info()

Типы данных до изменения:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   days_employed     21424 non-null  float64
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   education_id      21424 non-null  int64  
 5   family_status     21424 non-null  object 
 6   family_status_id  21424 non-null  int64  
 7   gender            21424 non-null  object 
 8   income_type       21424 non-null  object 
 9   debt              21424 non-null  int64  
 10  total_income      21424 non-null  float64
 11  purpose           21424 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Округлим значения столбцов с типом "float", так данные будут точнее при изменении типа.

In [22]:
data['days_employed'] = data['days_employed'].round()
data['total_income'] = data['total_income'].round()

In [23]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [24]:
print('Типы данных после изменения:')
data.info()

Типы данных после изменения:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21424 non-null  int64 
 1   days_employed     21424 non-null  int64 
 2   dob_years         21424 non-null  int64 
 3   education         21424 non-null  object
 4   education_id      21424 non-null  int64 
 5   family_status     21424 non-null  object
 6   family_status_id  21424 non-null  int64 
 7   gender            21424 non-null  object
 8   income_type       21424 non-null  object
 9   debt              21424 non-null  int64 
 10  total_income      21424 non-null  int64 
 11  purpose           21424 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Перевод данных из вещественного типа в целочисленный был произведен методом astype.

### Обработка дубликатов


In [25]:
print('Распределение детей по возростам:')
print(data['children'].value_counts())

Распределение детей по возростам:
 0     14080
 1      4802
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [26]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)

In [27]:
print('Гендеры:')
print(data['gender'].value_counts())

Гендеры:
F      14164
M       7259
XNA        1
Name: gender, dtype: int64


In [28]:
data = data.drop(data[data['gender'] == 'XNA'].index)

In [29]:
print('Уникальные значения столбца "образование":')
print(data['education'].value_counts())

Уникальные значения столбца "образование":
среднее                13691
высшее                  4686
СРЕДНЕЕ                  770
Среднее                  708
неоконченное высшее      665
ВЫСШЕЕ                   273
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [30]:
data['education'] = data['education'].str.lower()

In [31]:
print('Уникальные значения столбца family_status и их количество:')
print(data['family_status'].value_counts())

Уникальные значения столбца family_status и их количество:
женат / замужем          12331
гражданский брак          4155
Не женат / не замужем     2797
в разводе                 1185
вдовец / вдова             955
Name: family_status, dtype: int64


In [32]:
print('Уникальные значения столбца income_type и их количество:')
print(data['income_type'].value_counts())

Уникальные значения столбца income_type и их количество:
сотрудник          11064
компаньон           5064
пенсионер           3836
госслужащий         1453
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


In [33]:
print('Уникальные значения столбца debt и их количество:')
print(data['debt'].value_counts())  

Уникальные значения столбца debt и их количество:
0    19690
1     1733
Name: debt, dtype: int64


In [34]:
print('Уникальные значения столбца purpose и их количество:')
print(data['purpose'].value_counts()) 

Уникальные значения столбца purpose и их количество:
свадьба                                   792
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     649
операции с жильем                         647
жилье                                     641
покупка жилья                             641
покупка жилья для семьи                   640
строительство собственной недвижимости    633
недвижимость                              630
операции со своей недвижимостью           630
строительство жилой недвижимости          623
покупка недвижимости                      620
строительство недвижимости                620
покупка своего жилья                      619
ремонт жилью                              610
покупка жилой недвижимости                604
на покупку своего автомобил

In [35]:
print('Количество дубликатов:')
print(data.duplicated().sum())

Количество дубликатов:
71


In [36]:
data = data.drop_duplicates().reset_index(drop=True)

In [37]:
print('Финальная проверка на дубликаты:')
print(data.duplicated().sum())

Финальная проверка на дубликаты:
0


Вывод:

В данных были обнаружены определенные "артефакты". В случае со столбцом children, данные были изменены. Количество детей не может быть отрицательным. Так же было предположено, что число детей "20", это ошибка, данные в этих строках были измененым в пользу наиболее вероятного значения "2". Также была удалена строка с гендером XNA. Методом str.lower строковые данные столбца education были приведены к нижнему регистру. Явные дубликаты были удалены.

## Формирование новых признаков

### Лемматизация

In [38]:
print(data['purpose'].value_counts().keys())

Index(['свадьба', 'на проведение свадьбы', 'сыграть свадьбу',
       'операции с недвижимостью', 'покупка коммерческой недвижимости',
       'покупка жилья для сдачи', 'операции с коммерческой недвижимостью',
       'операции с жильем', 'жилье', 'покупка жилья',
       'покупка жилья для семьи', 'строительство собственной недвижимости',
       'недвижимость', 'операции со своей недвижимостью',
       'строительство жилой недвижимости', 'покупка своего жилья',
       'строительство недвижимости', 'покупка недвижимости', 'ремонт жилью',
       'покупка жилой недвижимости', 'на покупку своего автомобиля',
       'заняться высшим образованием', 'автомобиль',
       'сделка с подержанным автомобилем',
       'на покупку подержанного автомобиля', 'автомобили', 'свой автомобиль',
       'на покупку автомобиля', 'приобретение автомобиля',
       'дополнительное образование', 'сделка с автомобилем',
       'высшее образование', 'получение дополнительного образования',
       'образование', 'пол

Просматривая уникальныне значения столбца с целями получения кредита, можно заметить, что все цели сводятся к 4м различным категриям: опериции с недвижимостью (с жильем), получение кредита на свадьбу, на автомобиль и на образование.
С помощью стемминга найдем основу этих слов, проведем лемматизацию столбца purpose и напишем функцию по распределению значений стобца purpose по 4м категориям.


In [39]:
russian_stemmer = SnowballStemmer('russian') 
words = ["недвижимость", "жилье", "автомобиль", "свадьба", "образование"]
print('Стеминг:')
for word in words:
    print ('Исходное слово - ' + word + ', после стемминга - ' + russian_stemmer.stem(word)) 

Стеминг:
Исходное слово - недвижимость, после стемминга - недвижим
Исходное слово - жилье, после стемминга - жил
Исходное слово - автомобиль, после стемминга - автомобил
Исходное слово - свадьба, после стемминга - свадьб
Исходное слово - образование, после стемминга - образован


In [40]:
m = Mystem()
lemmas = data['purpose'].apply(m.lemmatize)


In [41]:
print(lemmas)

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21347                  [операция,  , с,  , жилье, \n]
21348               [сделка,  , с,  , автомобиль, \n]
21349                              [недвижимость, \n]
21350    [на,  , покупка,  , свой,  , автомобиль, \n]
21351             [на,  , покупка,  , автомобиль, \n]
Name: purpose, Length: 21352, dtype: object


In [42]:
def sort_purpose_lemmas(lemmas):
    for row in lemmas:
        if 'недвижим' in row:
            return 'недвижимость'
        elif 'жил' in row:
            return 'недвижимость'
        elif 'свадьб' in row:
            return 'свадьба'
        elif 'автомобил' in row:
            return 'автомобиль'
        elif 'образован' in row:
            return 'образование'
        

In [43]:
data['purpose_lemmas'] = lemmas.apply(sort_purpose_lemmas)


In [44]:
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,недвижимость
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,недвижимость
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21347,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224792,операции с жильем,недвижимость
21348,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,156000,сделка с автомобилем,автомобиль
21349,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89673,недвижимость,недвижимость
21350,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


**Вывод**

Лемматизация проводилась, используя библиотеку pymystem3, и заключалась в определении 4х различных целей получения кредита. Это недвижимость, автомобиль, образование и свадьба. В таблицу добавлен новый столбец с названием категории для цели получения кредита.

### Категоризация данных

In [45]:
print(data['total_income'].describe().round())

count      21352.0
mean      165350.0
std        98116.0
min        20667.0
25%       107649.0
50%       145905.0
75%       195760.0
max      2265604.0
Name: total_income, dtype: float64


Разбиение "дохода" на категории:

In [66]:
T = []
for raw in data['total_income']:
    if raw == 0:
        raw = 'доход не определен'
        T.append(raw)
    else:
        if raw >= 30000 and raw < 60000:
            raw = 'средний доход'
            T.append(raw) 
        else:
            if raw >= 60000 and raw < 100000:
                raw = 'доход выше среднего'
                T.append(raw)
            else:
                if raw >= 100000 and raw < 500000:
                    raw = 'высокий доход'
                    T.append(raw)
                else:
                    if raw >= 500000:
                        raw = 'очень высокий доход'
                        T.append(raw)
                    else:
                        raw = 'низкий доход'
                        T.append(raw) 

data['total_income_category'] = T

In [67]:
addiction_total_income_categor = data.pivot_table(
    index='total_income_category', 
    values='debt',
    aggfunc='mean'
    )

In [68]:
addiction_children = data.pivot_table(
    index='children', 
    values='debt',
    aggfunc='mean'
    )

In [69]:
addiction_family_status = data.pivot_table(
    index='family_status', 
    values='debt',
    aggfunc='mean'
    )

In [70]:
addiction_purpose_lemmas = data.pivot_table(
    index='purpose_lemmas', 
    values='debt',
    aggfunc='mean'
    )

Для категоризации данный по ежемесячному доходу, разобем значения этого столбца на 6 различных уровней дохода. Такое разделение обусловлено минимальным, максимальным, средним и медианным значением этого столбца. Значения равные нулю в этом столбце говорят о том, что уровень дохода заемщика, в данном случае, не определен. Добавим в таблицу столбец, характеризующий уровень дохода заемщика по категориям. Создадим сводные таблицы из данных наличия задолжности по кредиту.

**Вывод**

Чтобы проследить зависимоть возврата кредита в срок от уровня дохода заемщика, потребовалось поместить данные об уровне дохода каждого заемщика в условную категорию, характеризующую его доход. Далее методом pivot_table была посчитана зависимоть между доходом заемщика и количесмтвом наличия задолжностей в этой катеогрии. Такая же зависимость была посчитана между количеством детей и возвратом кредита, между цель взятия кредита и возвратом кредита, между семейным положением заемщика и возвратом кредита. 

## Установление зависимостей целевого признака от признаков данных 

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [72]:
display(addiction_children)

,debt
children,
0,0.075458
1,0.091341
2,0.095553
3,0.082317
4,0.097561
5,0.000000


**Вывод**

В целом наблюдается тенденция роста надежности заемщика с ростом количества детей. Заемщики без детей хуже всего возвращают кредит. Заемщик с количеством детей 5 не имеют задолжности по возврату кредита. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [73]:
display(addiction_family_status)

,debt
family_status,
Не женат / не замужем,0.097709
в разводе,0.071730
вдовец / вдова,0.064990
гражданский брак,0.093485
женат / замужем,0.075427


**Вывод**

Не женатые / не замужние люди хуже всего отдают кредиты. Далее по степени возврата идут заемщики, находящиеся в гражданском браке. Далее - женатые и замужние. Далее - заемщики, находящиеся в разводе. Чаще всего возвращают кредиты в срок вдовы / вдовцы. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [74]:
display(addiction_total_income_categor)

,debt
total_income_category,
высокий доход,0.081845
доход выше среднего,0.083562
низкий доход,0.090909
очень высокий доход,0.063063
средний доход,0.060256


**Вывод**

Люди со среднмм доходом и с очень высоким доходом (доход более 500000) чаще всего возвращают кредит. Самые худщие заемщики - люди с низким доходом (менее 30000). Люди с высоким доходом, с доходом выше среденего и люди, чей доход не определен имеют примерно одикаковый коэффициент возврата кредита, они склонны к невозврату кредита в срок. 

- Как разные цели кредита влияют на его возврат в срок?

In [75]:
display(addiction_purpose_lemmas)

,debt
purpose_lemmas,
автомобиль,0.093371
недвижимость,0.072378
образование,0.092616
свадьба,0.079654


**Вывод**

Кредит на недвижимость является самым возвращаемым кредитом. Далее по степени возврата идет кредит на свадьб, далее - кредит на образование. Хуже всего возвращают кредит, связанный с автомобилями. 

## Общий вывод


Главным вопросом анализа этой таблицы с данными банка о заемщиках был, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок? Обработав и проанадизировав данные, можно уверенно сказать, что возврат кредита в срок стоит практически в прямой зависимости от количества детей и семейного положение заемщика. Большое количество детей может служить определенным маркером надежности заемщика перед банком. Вот такие дела.